In [1]:
import pandas as pd

# 1-import Data


    ### -use concat to add the new data from other months in the future


In [2]:
taxi_jan_2019 = pd.read_parquet('/content/yellow_tripdata_2019-01.parquet')
taxi_db = pd.concat([taxi_jan_2019])

FileNotFoundError: [Errno 2] No such file or directory: 'data/yellow_tripdata_2019-01.parquet'

In [ ]:
taxi_db.head()

Take a look

In [ ]:
taxi_db.shape

# 2-Data exporation

In [ ]:
taxi_db.columns

In [ ]:
taxi_db = taxi_db[['tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID','PULocationID',
        'DOLocationID', 'payment_type', 'total_amount']]

In [ ]:
taxi_db.head()

In [ ]:
taxi_db.shape

In [ ]:
taxi_db.hist(figsize=(20,10), bins=60)

In [ ]:
taxi_db['RatecodeID'].value_counts()


In [ ]:
#taxi_db.reset_index().plot(kind='scatter', y='total_amount', x='index', figsize=(10,5))

In [ ]:
#taxi_db[taxi_db['total_amount']<1000].reset_index().plot(kind='scatter', y='total_amount', x='index', figsize=(10,6))

In [ ]:
print(taxi_db[taxi_db['total_amount']<0].shape)
#taxi_db[taxi_db['total_amount']<0].reset_index().plot(figsize=(10,5) ,kind='scatter', y='total_amount', x='index')

In [ ]:
taxi_db[taxi_db['total_amount']<0].head()

In [ ]:
taxi_db[taxi_db['total_amount']<0]['payment_type'].value_counts()

In [ ]:
taxi_db[taxi_db['total_amount']==0].shape
taxi_db[taxi_db['total_amount']==0].head()

In [ ]:
taxi_db_prepared = taxi_db.copy()

In [ ]:
taxi_db_prepared = taxi_db_prepared[(taxi_db_prepared['total_amount']>=0)&(taxi_db_prepared['total_amount']<2000)]
taxi_db_prepared.dtypes

In [ ]:
taxi_db_prepared['transiction_date'] = pd.to_datetime(taxi_db_prepared['tpep_pickup_datetime'].dt.date)
taxi_db_prepared['transiction_year'] = taxi_db_prepared['tpep_pickup_datetime'].dt.year
taxi_db_prepared['transiction_month'] = taxi_db_prepared['tpep_pickup_datetime'].dt.month
taxi_db_prepared['transiction_day'] = taxi_db_prepared['tpep_pickup_datetime'].dt.day
taxi_db_prepared['transiction_hour'] = taxi_db_prepared['tpep_pickup_datetime'].dt.hour

In [ ]:
taxi_db_prepared.head()

In [ ]:
#taxi_db_prepared.hist(figsize=(20,10), bins = 60)

In [ ]:
taxi_db_prepared = taxi_db_prepared[taxi_db_prepared['transiction_year'] == 2019]
taxi_db_prepared = taxi_db_prepared[taxi_db_prepared['transiction_month'] == 1]

In [ ]:
taxi_db_prepared.head()


In [ ]:
categioral_columns = ['PULocationID', 'transiction_date','transiction_month','transiction_day','transiction_hour']
numerical_columns = ['trip_distance','total_amount']
all_needed_columns = categioral_columns + numerical_columns



In [ ]:
main_taxi_db = taxi_db_prepared[all_needed_columns]
print(main_taxi_db.shape)
main_taxi_db.head()

In [ ]:
taxi_grouped_by_region = main_taxi_db.groupby(categioral_columns).mean().reset_index()
taxi_grouped_by_region['transiction_count'] = main_taxi_db.groupby(categioral_columns).count().reset_index()['total_amount']
print(taxi_grouped_by_region.shape)
taxi_grouped_by_region.head()

In [ ]:
taxi_grouped_by_region['total_amount'].hist(bins=100, figsize=(10,5))


In [ ]:
taxi_grouped_by_region['trip_distance'].hist(bins=100, figsize=(10,5))

In [ ]:
data_benchmark_model = taxi_grouped_by_region.copy()
data_benchmark_model

In [ ]:
categioral_features_benchmark = ['PULocationID','transiction_month','transiction_day','transiction_hour']
input_features_benchmark = categioral_features_benchmark + ['trip_distance']
target_features_benchmark = ['total_amount']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_bench = data_benchmark_model[input_features_benchmark]
y_bench = data_benchmark_model[target_features_benchmark]

x_bench = pd.get_dummies(x_bench)

x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(x_bench,y_bench, test_size= 0.33, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=10)
tree.fit(x_train_b,y_train_b)

In [ ]:
model_at_hand = tree

y_pred_b = model_at_hand.predict(x_test_b)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

print('mean_absolute_error',mean_absolute_error(y_test_b,y_pred_b))
print('mean_squared_error',mean_squared_error(y_test_b,y_pred_b))
print('sqrt',sqrt(mean_squared_error(y_test_b,y_pred_b)))
print('r2_score',r2_score(y_test_b, y_pred_b))